In [1]:
import os, math, numpy as np, pandas as pd
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from scipy import sparse

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

import pickle


## Save/load profiles

In [2]:
PROFILE_CACHE_DIR = "./cache_profiles"
os.makedirs(PROFILE_CACHE_DIR, exist_ok=True)

def save_profiles(profiles_dict, method_name):
    file_path = os.path.join(PROFILE_CACHE_DIR, f"profiles_{method_name}.pkl")
    with open(file_path, "wb") as f:
        pickle.dump(profiles_dict, f)

def load_profiles(method_name):
    file_path = os.path.join(PROFILE_CACHE_DIR, f"profiles_{method_name}.pkl")
    if os.path.exists(file_path):
        with open(file_path, "rb") as f:
            return pickle.load(f)
    return None

## Parameters

In [3]:
DATA_DIR = "../data"
Z_THRESHOLD = 0.0
RATING_MIN = 0.5
RATING_MAX = 5.0
TEST_SIZE = 0.10
VAL_SIZE = 0.10
SEED = 42
SAMPLE_USER = 72313
TOP_N = 10
SHOW_ROWS = 50
N_FEATURES_FS = 300

## Load data

In [4]:
ratings = pd.read_csv(f"{DATA_DIR}/ratings.csv")
movies = pd.read_csv(f"{DATA_DIR}/movies.csv")

genome_scores = pd.read_csv(f"{DATA_DIR}/genome-scores.csv")
genome_tags = pd.read_csv(f"{DATA_DIR}/genome-tags.csv")


In [5]:
df = pd.merge(ratings, movies, on='movieId')
display(df.head())
print(f"Dataset: {len(df):,} ratings | {df['userId'].nunique():,} users | {df['movieId'].nunique():,} movies")

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War
4,1,899,3.5,1147868510,Singin' in the Rain (1952),Comedy|Musical|Romance


Dataset: 25,000,095 ratings | 162,541 users | 59,047 movies


## Per‑user z‑score (remove bias)

Mỗi người dùng có thang chấm điểm khác nhau. Ví dụ, một người có thể chấm 4/5 là "rất thích", trong khi người khác chấm 4/5 là "bình thường". Chuẩn hóa z-score giúp so sánh đánh giá giữa các người dùng một cách công bằng.

In [6]:
user_stats = df.groupby("userId")["rating"].agg(["mean", "std"]).rename(columns={"mean": "mu", "std": "sigma"})
user_stats

,mu,sigma
userId,,
1,3.814286,1.004235
2,3.630435,1.457728
3,3.697409,0.599854
4,3.378099,1.116927
5,3.752475,0.931729
...,...,...
162537,4.039604,0.958340
162538,3.415584,1.216452
162539,4.510638,0.718463


**Z-score:** `z = (x - mu) / sigma`, trong đó `x` là đánh giá, `mu` là trung bình đánh giá của người dùng, `sigma` là độ lệch chuẩn.

- Z-score biểu thị đánh giá lệch bao nhiêu so với trung bình của người dùng, chuẩn hóa về đơn vị độ lệch chuẩn.

- Nếu `sigma=0` (người dùng chỉ chấm một giá trị duy nhất), phép chia sẽ gây lỗi (chia cho 0). Thay bằng 1e-6 để tránh lỗi và giữ z-score hợp lý.

In [7]:
df = df.join(user_stats, on="userId")
df["rating_z"] = (df["rating"] - df["mu"]) / df["sigma"].replace(0, 1e-6)
df.head()

,userId,movieId,rating,timestamp,title,genres,mu,sigma,rating_z
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,3.814286,1.004235,1.180714
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama,3.814286,1.004235,-0.312960
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama,3.814286,1.004235,1.180714
3,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War,3.814286,1.004235,1.180714
4,1,899,3.5,1147868510,Singin' in the Rain (1952),Comedy|Musical|Romance,3.814286,1.004235,-0.312960


In [8]:
df.describe()

,userId,movieId,rating,timestamp,mu,sigma,rating_z
count,2.500010e+07,2.500010e+07,2.500010e+07,2.500010e+07,2.500010e+07,2.500010e+07,2.500010e+07
mean,8.118928e+04,2.138798e+04,3.533854e+00,1.215601e+09,3.533854e+00,9.193482e-01,-3.058164e-19
std,4.679172e+04,3.919886e+04,1.060744e+00,2.268758e+08,4.784993e-01,2.388860e-01,9.964306e-01
min,1.000000e+00,1.000000e+00,5.000000e-01,7.896520e+08,5.000000e-01,0.000000e+00,-2.824892e+01
25%,4.051000e+04,1.196000e+03,3.000000e+00,1.011747e+09,3.250000e+00,7.540528e-01,-6.097788e-01
50%,8.091400e+04,2.947000e+03,3.500000e+00,1.198868e+09,3.552746e+00,9.008775e-01,1.105046e-01
75%,1.215570e+05,8.623000e+03,4.000000e+00,1.447205e+09,3.848468e+00,1.063137e+00,7.170283e-01
max,1.625410e+05,2.091710e+05,5.000000e+00,1.574328e+09,5.000000e+00,2.308451e+00,3.459683e+01


## Encode

Các `userId` và `movieId` có thể không liên tục (ví dụ: 1, 3, 7). Mã hóa thành chỉ số liên tục (0, 1, 2, ...) giúp dễ xử lý trong ma trận và tiết kiệm bộ nhớ.

In [9]:
user_encoder = LabelEncoder()
#movie_encoder = LabelEncoder()

user_encoder.fit(df['userId'])
#movie_encoder.fit(movies['movieId'])

df['user_idx'] = user_encoder.transform(df['userId'])
#df['movie_idx'] = movie_encoder.transform(df['movieId'])

df.head()

,userId,movieId,rating,timestamp,title,genres,mu,sigma,rating_z,user_idx
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,3.814286,1.004235,1.180714,0
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama,3.814286,1.004235,-0.312960,0
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama,3.814286,1.004235,1.180714,0
3,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War,3.814286,1.004235,1.180714,0
4,1,899,3.5,1147868510,Singin' in the Rain (1952),Comedy|Musical|Romance,3.814286,1.004235,-0.312960,0


In [10]:
n_users = df["user_idx"].nunique()
n_items = df["movieId"].nunique()
print(f"Dataset: {len(df):,} ratings | {n_users:,} users | {n_items:,} movies")

Dataset: 25,000,095 ratings | 162,541 users | 59,047 movies


- Train: Dùng để xây dựng hồ sơ người dùng (user profile).

- Validation: Đánh giá hiệu suất mô hình trong quá trình phát triển, điều chỉnh tham số (như `Z_THRESHOLD`).

- Test: Đánh giá cuối cùng để báo cáo hiệu suất thực tế.

`stratify` đảm bảo rằng tỉ lệ người dùng trong các tập train, validation, test tương tự nhau. Điều này quan trọng vì mỗi người dùng có số lượng đánh giá khác nhau, và ta muốn mô hình được huấn luyện trên dữ liệu đại diện.

In [11]:
split_dir = os.path.join(DATA_DIR, "splits")
os.makedirs(split_dir, exist_ok=True)

train_path = os.path.join(split_dir, "train.csv")
val_path   = os.path.join(split_dir, "val.csv")
test_path  = os.path.join(split_dir, "test.csv")

if os.path.exists(train_path) and os.path.exists(val_path) and os.path.exists(test_path):
    print("Loading cached train/val/test splits...")
    train_df = pd.read_csv(train_path)
    val_df   = pd.read_csv(val_path)
    test_df  = pd.read_csv(test_path)
else:
    print("Splitting train/val/test...")
    def train_val_test_plit(
        data_frame,
        test_size=TEST_SIZE,
        val_size=VAL_SIZE,
        col="user_idx",
        seed=SEED
    ):
        train_val, test = train_test_split(
            data_frame,
            test_size=test_size,
            random_state=seed,
            stratify=data_frame[col]
        )
        train, val = train_test_split(
            train_val,
            test_size=val_size,
            random_state=seed,
            stratify=train_val[col]
        )
        return train.reset_index(drop=True), val.reset_index(drop=True), test.reset_index(drop=True)

    train_df, val_df, test_df = train_val_test_plit(df)

    train_df.to_csv(train_path, index=False)
    val_df.to_csv(val_path, index=False)
    test_df.to_csv(test_path, index=False)

print("Split sizes -", {k: len(v) for k, v in zip(["train", "val", "test"], [train_df, val_df, test_df])})

Splitting train/val/test...


KeyboardInterrupt: 

## Build genome CSR matrix

Mỗi phim được biểu diễn bằng một vector các điểm `relevance` tương ứng với các tag. Ta cần tạo một ma trận **phim × tag**, trong đó mỗi ô là điểm `relevance`.

In [ ]:
merged_genome = pd.merge(genome_scores, genome_tags, on='tagId', how='left')  
#merged_genome['movie_idx'] = movie_encoder.transform(merged_genome['movieId'])

merged_genome

In [ ]:
genome_matrix = merged_genome.pivot(index='movieId', columns='tag', values='relevance').fillna(0)
genome_matrix

In [ ]:
# movieId2row = dict(zip(genome_matrix.index.values, np.arange(genome_matrix.shape[0], dtype=np.int32)))
# row2movieId = genome_matrix.index.values

## Genres Vectorization (TF-IDF)

In [ ]:
tfidf = TfidfVectorizer(token_pattern=r'[^|]+')
genres_tfidf = tfidf.fit_transform(movies['genres'])
genres_df = pd.DataFrame(genres_tfidf.toarray(), columns=[f'genre:{g}' for g in tfidf.get_feature_names_out()],
                                                    index=movies['movieId'])

In [ ]:
describe_matrix = pd.concat([genome_matrix, genres_df], axis=1).fillna(0)
describe_matrix

In [ ]:
std_scaler = StandardScaler(with_mean=False)
describe_matrix_scaled = std_scaler.fit_transform(describe_matrix) 
print(describe_matrix_scaled)

## Feature Selection

In [ ]:
# Tính rating trung bình cho mỗi movieId
movie_avg_rating = ratings.groupby('movieId')['rating'].mean()

# Lọc các movieId hợp lệ có trong genome data
genome_df_filtered = merged_genome[merged_genome['movieId'].isin(movie_avg_rating.index)].copy()

# Xoá trùng movieId nếu cần (giữ lại 1 dòng duy nhất)
genome_df_filtered = genome_df_filtered.drop_duplicates(subset='movieId')

# Chỉ giữ lại cột movieId
genome_df_filtered = genome_df_filtered[['movieId']].copy().reset_index(drop=True)

# Thêm cột trung bình rating
genome_df_filtered['avg_rating'] = genome_df_filtered['movieId'].map(movie_avg_rating)

In [ ]:
genome_df_filtered.head()

In [ ]:
def rating_to_class(rating):
    if rating <= 2.5:
        return 0  # rating thấp
    elif rating <= 4.0:
        return 1  # rating trung bình
    else:
        return 2  # rating cao

rating_class = movie_avg_rating.apply(rating_to_class)

In [ ]:
y_sup = rating_class.reindex(describe_matrix.index).fillna(1).astype(int).values

if sparse.issparse(describe_matrix_scaled):
    X_dense = describe_matrix_scaled.toarray()
    X_chi = MinMaxScaler().fit_transform(describe_matrix_scaled)  
else:
    X_dense = describe_matrix_scaled        
    X_chi = MinMaxScaler().fit_transform(describe_matrix_scaled)  


# tạo 4 ma trận đặc trưng giám sát
feature_matrices = {}

# 1. Chi-squared
sel_chi = SelectKBest(chi2, k=N_FEATURES_FS).fit(X_chi, y_sup)
feature_matrices['chi2'] = sel_chi.transform(describe_matrix_scaled)

# 2. Information Gain (Entropy)
tree_e = DecisionTreeClassifier(criterion='entropy', random_state=0).fit(X_dense, y_sup)
idx_e = np.argsort(tree_e.feature_importances_)[-N_FEATURES_FS:]
feature_matrices['entropy'] = X_dense[:, idx_e]

# 3. Gini Index
tree_g = DecisionTreeClassifier(criterion='gini', random_state=0).fit(X_dense, y_sup)
idx_g = np.argsort(tree_g.feature_importances_)[-N_FEATURES_FS:]
feature_matrices['gini'] = X_dense[:, idx_g]

# 4. Normalized Deviation
std_col = np.std(X_dense, axis=0)
idx_std = np.argsort(std_col)[-N_FEATURES_FS:]
feature_matrices['deviation'] = X_dense[:, idx_std]

## Build user profiles

**Làm thế nào để biểu diễn sở thích của người dùng?**

- Hồ sơ người dùng là một vector tổng hợp các tag genome của những phim họ thích (dựa trên `rating_z ≥ Z_THRESHOLD`). Vector này được tính bằng trung bình có trọng số của các vector phim.

In [ ]:
def build_user_profiles(ratings_df, rating_col="rating_z", threshold=Z_THRESHOLD, eps=1e-8):
    profiles = {}
    good = ratings_df.loc[ratings_df[rating_col] >= threshold, ["userId", "movieId", rating_col]]
    
    for uid, grp in tqdm(good.groupby("userId"), desc="profiles", unit="user"):
        rows = [movieId2row[m] for m in grp.movieId if m in movieId2row]
        if not rows:
            continue

        w = grp[rating_col].values[:, None]
        
        if w.shape[0] == len(rows):
            w_sum = w.sum()

            if w_sum < eps:
                prof_dense = genome_csr[rows].mean(axis=0)
            else:
                prof_dense = (genome_csr[rows].multiply(w)).sum(axis=0) / w_sum

            prof_dense = np.nan_to_num(np.asarray(prof_dense).ravel())
            profiles[uid] = sparse.csr_matrix(prof_dense)

    return profiles

In [ ]:
def content_score(uid, mid):
    if uid not in profiles or mid not in movieId2row:
        return 0.0
    vec = profiles[uid]
    if vec.nnz == 0 or np.isnan(vec.data).any():
        return 0.0
    return float(cosine_similarity(vec, genome_csr[movieId2row[mid]])[0, 0])

def scale_to_rating(sim, a=0, b=1, c=RATING_MIN, d=RATING_MAX):
    return (sim - a) / (b - a) * (d - c) + c

## Evaluation

In [ ]:
def evaluate(df_subset):
    y_true = df_subset["rating"].values
    y_pred = [scale_to_rating(content_score(u, m)) for u, m in zip(df_subset["userId"], df_subset["movieId"])]
    return math.sqrt(mean_squared_error(y_true, y_pred)), mean_absolute_error(y_true, y_pred)

def _test_matrix(mat):
    global genome_csr, movieId2row, row2movieId, profiles
    genome_csr = normalize(sparse.csr_matrix(mat), axis=1, copy=False)
    movieId2row = {mid: i for i, mid in enumerate(describe_matrix.index.values)}
    row2movieId = describe_matrix.index.values
    profiles = build_user_profiles(train_df)
    return evaluate(test_df)

In [ ]:
results = {}
print("----------Đánh giá 4 phương pháp chọn đặc trưng----------")
for name, mat in feature_matrices.items():
    rmse, mae = _test_matrix(mat)
    results[name] = (rmse, mae)
    print(f"{name:<9}: RMSE = {rmse:.4f} | MAE = {mae:.4f}")

In [ ]:
best_method = min(results, key=lambda k: results[k][0])
print(f"Best method: {best_method}  (RMSE = {results[best_method][0]:.4f})")

In [ ]:
genome_csr = normalize(sparse.csr_matrix(feature_matrices[best_method]), axis=1, copy=False)

movieId2row = {mid: i for i, mid in enumerate(describe_matrix.index.values)}
row2movieId = describe_matrix.index.values

In [ ]:
movieId2row

In [ ]:
row2movieId

In [ ]:
profiles = build_user_profiles(train_df) 

In [ ]:
# Cập nhật genome_csr theo phương pháp tốt nhất
genome_csr = normalize(sparse.csr_matrix(feature_matrices[best_method]), axis=1, copy=False)

# Cập nhật chỉ mục movieId
movieId2row = {mid: i for i, mid in enumerate(describe_matrix.index.values)}
row2movieId = describe_matrix.index.values

# Load hoặc build user profiles
profiles = load_profiles(best_method)
if profiles is None:
    print(f"Building user profiles for method: {best_method}")
    profiles = build_user_profiles(train_df)
    save_profiles(profiles, best_method)
    print(f"Profiles saved to disk for method '{best_method}'")
else:
    print(f"Loaded cached profiles for method '{best_method}'")


## Recommendation

In [ ]:
def print_actual_pred(df_, n=SHOW_ROWS):
    print(f"Actual vs Predicted (first {n} rows of supplied set):")
    for _, row in df_.head(n).iterrows():
        pred = scale_to_rating(content_score(row.userId, row.movieId))
        print(f"userid = {row.userId:6.0f} | movieid = {row.movieId:6.0f} | actual = {row.rating:3.1f} | pred = {pred:3.2f}")

print_actual_pred(test_df, SHOW_ROWS)

In [ ]:
def recommend_content(uid, N=TOP_N):
    if uid not in profiles or not profiles[uid].nnz:
        return []
    seen = set(train_df.loc[train_df.userId == uid, "movieId"])
    sims = cosine_similarity(profiles[uid], genome_csr).ravel()
    for m in seen:
        if m in movieId2row:
            sims[movieId2row[m]] = -1.0
    idx = np.argpartition(sims, -N)[-N:]
    idx = idx[np.argsort(sims[idx])[::-1]]
    return [(int(row2movieId[i]), float(sims[i])) for i in idx]

def user_genre_preference(uid, top_k=10):
    fav = train_df[(train_df.userId == uid) & (train_df["rating_z"] >= Z_THRESHOLD)]
    counts = {}
    for mid in fav.movieId:
        for g in movies.loc[movies.movieId == mid, "genres"].iat[0].split("|"):
            counts[g] = counts.get(g, 0) + 1
    return dict(sorted(counts.items(), key=lambda kv: kv[1], reverse=True)[:top_k])

def show_recs(uid=SAMPLE_USER, N=TOP_N):
    print(f"\nTop-{N} recommendations for user {uid}")
    for mid, sim in recommend_content(uid, N):
        mv = movies[movies.movieId == mid].iloc[0]
        print(f"movieid = {mid:6d} | similarity = {sim:.3f} | predicted rating = {scale_to_rating(sim):.2f} | {mv.title} | {mv.genres}")
    print("\nUser's favourite genres:")
    for g, c in user_genre_preference(uid).items():
        print(f"{g}: {c} movies")

In [ ]:
if __name__ == "__main__":
    show_recs(SAMPLE_USER, TOP_N)